This code is based on the paper provided by Stiennon et al. that shows how to use PPO to train a LM using human feedback for a summarization task. 

# IMPORTS 

In [ ]:
from datasets import load_dataset
import os, math, random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM


# LOADING SFT MODEL

In [ ]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


policy_model = AutoModelForCausalLM.from_pretrained(model_name)
policy_model_ref = AutoModelForCausalLM.from_pretrained(model_name)

# LOADING REDDIT DATASET

In [ ]:
ds = load_dataset("openai/summarize_from_feedback", "comparisons")